TPU 사용을 위한 준비(GPU 쓸거면 안해도 됨.)

In [ ]:
# TPU 초기화
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

# 훈련을 여러 GPU, TPU등 여러 장비로 나누어 처리하기 위한 텐서플로 API
strategy = tf.distribute.TPUStrategy(resolver)

# 딥러닝 모델 컴파일을 위한 세팅
# 모델의 층을 쌓음
def create_model():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
       tf.keras.layers.Conv2D(256, 3, activation='relu'),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(256, activation='relu'),
       tf.keras.layers.Dense(128, activation='relu'),
       tf.keras.layers.Dense(10)])

In [ ]:
# 이 스코프 안에서 모델을 컴파일
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

transformers가 제공하는 BERT+출력층 모델 클래스 구현체가 있어 출력층을 직접 만들지 않아도 됨.

시맨틱 검색(Semantic search)은 기존의 키워드 매칭이 아닌 문장의 의미에 초점을 맞춘 정보 검색 시스템을 말합니다. SBERT와 FAISS를 사용하여 간단한 검색 엔진을 구현해봅시다.

- Faiss는 vector 유사도를 효율적으로 측정하는 라이브러리. 
- 벡터화 된 데이터를 인덱싱하고 데이터에 대한 효율적인 검색을 수행하기 위해 Facebook AI에서 구축한 C ++ 기반 라이브러리입니다. 
    
    
    CPU 환경에서 진행한다면 faiss-gpu가 아니라 faiss-cpu를 설치.

In [ ]:
!pip install faiss-gpu
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=1fe0b84367e15014a22493206d37cc008052798cd0ceb44d03fc282294c2b6d1
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e5

1. 데이터 로드

In [ ]:
import numpy as np
import os
import pandas as pd
import urllib.request
import faiss
import time
from sentence_transformers import SentenceTransformer

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/19.%20Topic%20Modeling%20(LDA%2C%20BERT-Based)/dataset/abcnews-date-text.csv", filename="abcnews-date-text.csv")

df = pd.read_csv("abcnews-date-text.csv")
data = df.headline_text.to_list()

# 상위 5개의 샘플 출력
data[:5]

['aba decides against community broadcasting licence',
 'act fire witnesses must be aware of defamation',
 'a g calls for infrastructure protection summit',
 'air nz staff in aust strike for pay rise',
 'air nz strike to affect australian travellers']

In [ ]:
print('총 샘플의 개수 :', len(data))

총 샘플의 개수 : 1082168


2. SBERT 임베딩

In [ ]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoded_data = model.encode(data)
print('임베딩 된 벡터 수 :', len(encoded_data))

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

임베딩 된 벡터 수 : 1082168


- 8분 걸림.

3. 인덱스 정의 및 데이터 추가

In [ ]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(data))))

faiss.write_index(index, 'abc_news')

4. 검색 및 시간 측정

In [ ]:
def search(query):
   t = time.time()
   query_vector = model.encode([query])
   k = 5
   top_k = index.search(query_vector, k)
   print('total time: {}'.format(time.time() - t))
   return [data[_id] for _id in top_k[1].tolist()[0]]

In [ ]:
query = str(input())
results = search(query)

print('results :')
for result in results:
   print('\t', result)

Underwater Forest Discovered
total time: 1.209993600845337
results :
	 underwater loop
	 thriving underwater antarctic garden discovered
	 baton goes underwater in wa
	 underwater footage shows inside doomed costa
	 underwater uluru found off wa coast
